In [52]:
import pandas as pd

# Load the existing CSV file
file_path = "template-SI.csv"  # Update the path if needed
df = pd.read_csv(file_path)


In [53]:
# Import Air temperature and Air speed from "HOBO/Group3-Low.csv"
hobo_low = "HOBO/Group3-Low.csv"
#hobo_df = pd.read_csv(hobo_file)
hobo_df_low = pd.read_csv(hobo_low, encoding="latin1")  # Try "ISO-8859-1" or "cp1252" if needed

hobo_mid = "HOBO/Group3-Mid.csv"
#hobo_df = pd.read_csv(hobo_file)
hobo_df_mid = pd.read_csv(hobo_mid, encoding="latin1")  # Try "ISO-8859-1" or "cp1252" if needed

hobo_top = "HOBO/Group3-Top.csv"
#hobo_df = pd.read_csv(hobo_file)
hobo_df_top = pd.read_csv(hobo_top, encoding="latin1")  # Try "ISO-8859-1" or "cp1252" if needed

# Import Relative Humidity values from "Relative Humidity.csv"
relative_humidity_file = "Relative Humidity.xlsx"  # Update the file name
rh_df = pd.read_excel(relative_humidity_file)  # Read the Excel file


In [54]:
# Determine the max number of rows required
#max_rows = max(hobo_df_low.count(), hobo_df_mid.count(), hobo_df_top.count(), rh_df.count())
max_rows = max(hobo_df_low.count().max(), 
               hobo_df_mid.count().max(), 
               hobo_df_top.count().max(), 
               rh_df.count().max())

# Expand df to match max_rows
df = df.reindex(range(max_rows)).reset_index(drop=True)

#print(max_rows)

In [55]:
# Expand DataFrames to match max_rows
hobo_df_low = hobo_df_low.reindex(range(max_rows))
hobo_df_mid = hobo_df_mid.reindex(range(max_rows))
hobo_df_top = hobo_df_top.reindex(range(max_rows))

# Determine the number of missing rows
missing_rows = max_rows - len(rh_df)
#print("rh_df_1", rh_df)

# Create a DataFrame filled with 48% for the missing rows
# Ensure missing_rh has the same columns as rh_df
missing_rh = pd.DataFrame({col: [48] * (max_rows - len(rh_df)) for col in rh_df.columns})

# Concatenate and trim to max_rows
rh_df = pd.concat([missing_rh, rh_df], ignore_index=True).iloc[:max_rows]


#if missing_rows > 0:
#    missing_rh = pd.DataFrame({"Relative humidity [%]": [48] * missing_rows})
    #print("missing rh", missing_rh)
#    rh_df = pd.concat([missing_rh, rh_df], ignore_index=True)

#print("rh_df_2", rh_df)
# Ensure final DataFrame has the correct number of rows
rh_df = rh_df.reindex(range(max_rows))



rh_df_1                Date Time   RH, %
0    2025-03-14 10:25:19  46.581
1    2025-03-14 10:25:20  46.615
2    2025-03-14 10:25:21  46.647
3    2025-03-14 10:25:22  46.710
4    2025-03-14 10:25:23  46.744
...                  ...     ...
3036 2025-03-14 11:15:55  33.983
3037 2025-03-14 11:15:56  33.983
3038 2025-03-14 11:15:57  33.983
3039 2025-03-14 11:15:58  33.983
3040 2025-03-14 11:15:59  33.983

[3041 rows x 2 columns]
missing rh       Relative humidity [%]
0                        48
1                        48
2                        48
3                        48
4                        48
...                     ...
4155                     48
4156                     48
4157                     48
4158                     48
4159                     48

[4160 rows x 1 columns]
rh_df_2       Relative humidity [%]           Date Time   RH, %
0                      48.0                 NaT     NaN
1                      48.0                 NaT     NaN
2                      

In [ ]:
# Assign to df
df["Relative humidity [%]"] = rh_df["RH, %"].values

# Compute the average values for Air Temperature and Air Speed
df["Air temperature [C]"] = (
    hobo_df_low["T_air (°C)"] + hobo_df_mid["T_air (°C)"] + hobo_df_top["T_air (°C)"]
) / 3

df["Air speed [m/s]"] = (
    hobo_df_low["Air_speed (m/s)"] + hobo_df_mid["Air_speed (m/s)"] + hobo_df_top["Air_speed (m/s)"]
) / 3

# Fill missing values if necessary
#df.fillna(method="ffill", inplace=True)  # Forward-fill missing data with the last value
df.fillna(0, inplace=True) # Forward-fill missing data with 0

# Constants for metabolic rate and clothing level
df["Metabolic rate [met]"] = 1.0
df["Clothing level [clo]"] = 0.57

In [44]:
# Verfication: Print the first 5 rows of Air temperature and Air speed --> OK
#print(df[["Air temperature [C]", "Air speed [m/s]"]].head())

# Verfication: Print the first 5 rows of Relative Humidity --> OK
#print(df["Relative humidity [%]"].head())

# Verfication: Print the first 5 rows of Relative Humidity --> OK
#print(df["MRT [C]"].head())

# Verfication: Print the first 5 rows of Metabolic rate and Clothing --> OK
#print(df[["Metabolic rate [met]", "Clothing level [clo]"]].head())

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Relative humidity [%], dtype: float64


In [25]:
# Print the number of rows for each column in the DataFrame
print("Number of rows per column in the main DataFrame:")
print(df.count())

print("\nNumber of rows per column in the HOBO DataFrame:")
print(hobo_df.count())

print("\nNumber of rows per column in the Relative Humidity DataFrame:")
print(rh_df.count())


Number of rows per column in the main DataFrame:
Air temperature [C]      7201
MRT [C]                  7201
Air speed [m/s]          7201
Relative humidity [%]    7201
Metabolic rate [met]     7201
Clothing level [clo]     7201
dtype: int64

Number of rows per column in the HOBO DataFrame:
Date & Time        7201
T_air (°C)         7201
T_globe (°C)       7201
Air_speed (m/s)    7201
dtype: int64

Number of rows per column in the Relative Humidity DataFrame:
Date Time    3041
RH, %        3041
dtype: int64


In [ ]:
# Save the updated CSV
#output_file_path = "updated_template-SI.csv"
#df.to_csv(output_file_path, index=False)

#print(f"CSV file updated and saved as {output_file_path}")